In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import os, glob, cv2, time
import numpy as np
import pytesseract
from PIL import Image

# pytesseract.pytesseract.tesseract_cmd = r"D:\Kuliah\Semester 1\Gamaforce\Repo\OCR-Python-master\Tesseract\tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def get_string(img_path):
    # Read image with opencv
    img = cv2.imread(img_path)

    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)

    # Write image after removed noise
    cv2.imwrite("removed_noise.png", img)

    #  Apply threshold to get image with only black and white
    #img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

    # Write the image after apply opencv to do some ...
    cv2.imwrite(img_path, img)

    # Recognize text with tesseract for python
    result = pytesseract.image_to_string(Image.open(img_path))

    # Remove template file
    #os.remove(temp)

    return result

def captcha():
    # web screenshot
    output = 'output.png'
    browser.save_screenshot(output)

    # img to numpy array
    capt = cv2.imread(output)
    capt = np.array(capt)

    # crop
    capt_crop = capt[ 130:160, 780:865, :]
    cv2.imwrite('output.png', capt_crop)
    # plt.imshow(capt_crop)

    # image to string OCR
    capt_str = get_string('output.png')

    # filter int from string
    capt_num = ''
    numbers = [str(word) for word in capt_str if word.isdigit()]
    capt_num = capt_num.join(numbers)

    # fill captcha
    capt_form = browser.find_element_by_xpath('//*[@id="grid-input-2"]')
    capt_form.send_keys(capt_num)

    # click submit buttom
    submit_but = browser.find_element_by_xpath('//*[@id="f_captcha"]/div[2]/div[1]/button[1]')
    submit_but.click()

    time.sleep(1)

    # accept alert
    alert = browser.switch_to.alert
    alert.accept()




Buka Chrome Webdriver

In [ ]:
chrome_options = Options()
chrome_options.binary_location  = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
browser = webdriver.Chrome(chrome_options = chrome_options, executable_path=r"C:\webdriver\chromedriver_win32\chromedriver.exe")

browser.get("https://simaster.ugm.ac.id")

Auto click + captcha solver

In [ ]:
id = [['TKSS212202',2], ['TKSS212204',2], ['TKSS212207',2], ['TKSS212208',2], ['TKSS212209',2], ['TKSS213209',1]]

while True:
    for matkul, kelas in id:
        print(matkul)

        browser.execute_script("arguments[0].click();", WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="8_{}"]/table[1]/tbody[1]/tr[{}]/td[1]/input[1]'.format(matkul, kelas)))))
        time.sleep(1)
        captcha()

        time.sleep(1)

In [ ]:
# close chrome webdriver
browser.close()